In [2]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/CS-198-199/filipino-fake-news/full.csv" "tl-full.csv"

Mounted at /content/drive


In [1]:
import pandas as pd
import itertools
import string
import re

In [ ]:
df = pd.read_csv('tl-full.csv', nrows=100) # first 100 rows only for testing
#print(df['article'])

## Simple Negate Rule #1:
#### sa -> sa hindi
#### sa mga -> sa mga hindi

In [13]:
def SimpleNegateSA(article):
  split_string = re.split(r'(?i)((?<!hindi)(?<!wala)(?<!para)\s+sa mga\s+(?!hindi|wala|walang|may)|(?<!hindi)(?<!wala)(?<!para)\s+sa\s+(?!mga|hindi|wala|walang|may))', article)
  print(split_string)

  for i in range(len(split_string)):
    if (split_string[i].casefold() == ' sa ') or (split_string[i].casefold() == ' sa mga '):
      split_string[i] = split_string[i] + "hindi "
  return "". join(split_string)

test_string = "Sinabi ni Cielo Villaluna, tagapagsalita ng PAL, ang bayarin sa mga terminal para sa international flights ay bahagi ng gastos sa mga tiket bilang pagsunod sa memorandum Agreement na nilagdaan ng Manila International Airport Authority (MIAA) at ng mga kumpanya ng airline sa bansa."
test_new = SimpleNegateSA(test_string)
print(test_new)

['Sinabi ni Cielo Villaluna, tagapagsalita ng PAL, ang bayarin', ' sa mga ', 'terminal para sa international flights ay bahagi ng gastos', ' sa mga ', 'tiket bilang pagsunod', ' sa ', 'memorandum Agreement na nilagdaan ng Manila International Airport Authority (MIAA) at ng mga kumpanya ng airline', ' sa ', 'bansa.']
Sinabi ni Cielo Villaluna, tagapagsalita ng PAL, ang bayarin sa mga hindi terminal para sa international flights ay bahagi ng gastos sa mga hindi tiket bilang pagsunod sa hindi memorandum Agreement na nilagdaan ng Manila International Airport Authority (MIAA) at ng mga kumpanya ng airline sa hindi bansa.


## Simple Negate Rule #2:
#### ay/ay ang -> ay hindi
#### ay mga/ay ang mga -> ay hindi mga
#### ay nasa -> ay wala sa

In [11]:
def SimpleNegateAY(article):
  split_string = re.split(r'(?i)(\s+ay ang mga\s+(?!hindi|wala|walang|nasa|may)|\s+ay nasa\s+(?!hindi|wala|walang)|\s+ay mga\s+(?!hindi|wala|walang|nasa|may)|\s+ay ang\s+(?!mga|hindi|wala|walang|nasa|may)|\s+ay\s+(?!ang mga|mga|hindi|wala|walang|nasa|may))', article)
  print(split_string)

  for i in range(len(split_string)):
    if (split_string[i].casefold() == ' ay ') or (split_string[i].casefold() == ' ay ang '):
        split_string[i] = " ay hindi "
    elif (split_string[i].casefold() == ' ay mga ') or (split_string[i].casefold() == ' ay ang mga '):
      split_string[i] = " ay hindi mga "
    elif (split_string[i].casefold() == ' ay nasa '):
      split_string[i] = " ay wala sa "
  return "". join(split_string)

test_string = "Ang suklay ay nasa bag niya. Ang concert ay sa Sabado na gaganapin. Kalahati nito ay mga ini-release na para sa mga paghahanda at pagsasaayos ng mga pasilidad para sa naturang Palaro. Ito ay ang nagsisilbing gabay para sa ating lahat. Ang nakitang nabasag ay ang mga baso."
test_new = SimpleNegateAY(test_string)
print(test_new)

['Ang suklay', ' ay nasa ', 'bag niya. Ang concert', ' ay ', 'sa Sabado na gaganapin. Kalahati nito', ' ay mga ', 'ini-release na para sa mga paghahanda at pagsasaayos ng mga pasilidad para sa naturang Palaro. Ito', ' ay ang ', 'nagsisilbing gabay para sa ating lahat. Ang nakitang nabasag', ' ay ang mga ', 'baso.']
Ang suklay ay wala sa bag niya. Ang concert ay hindi sa Sabado na gaganapin. Kalahati nito ay hindi mga ini-release na para sa mga paghahanda at pagsasaayos ng mga pasilidad para sa naturang Palaro. Ito ay hindi nagsisilbing gabay para sa ating lahat. Ang nakitang nabasag ay hindi mga baso.


## Simple Negate Rule #3:
#### may -> walang
#### mayroon -> wala
#### mayroong -> walang

In [6]:
def SimpleNegateMAY(article):
  split_string = re.split(r'(?i)(\s+mayroong\s+(?!hindi|wala|walang)|\s+mayroon\s+|\s+may\s+(?!hindi|wala|walang))', article)
  print(split_string)

  for i in range(len(split_string)):
    if (split_string[i].casefold() == ' may ') or (split_string[i].casefold() == ' mayroong '):
      split_string[i] = " walang "
    elif (split_string[i].casefold() == ' mayroon '):
      split_string[i] = " wala "
  return "". join(split_string)

test_string = "Siya ay may gustong sabihin. Mayroon bang nakapagdala ng pagkain? Ang kanilang grupo ay mayroong ipepresenta ngayon. Sila ay may hindi pagkakaunawaan."
test_new = SimpleNegateMAY(test_string)
print(test_new)

['Siya ay', ' may ', 'gustong sabihin.', ' Mayroon ', 'bang nakapagdala ng pagkain? Ang kanilang grupo ay', ' mayroong ', 'ipepresenta ngayon. Sila ay may hindi pagkakaunawaan.']
Siya ay walang gustong sabihin. wala bang nakapagdala ng pagkain? Ang kanilang grupo ay walang ipepresenta ngayon. Sila ay may hindi pagkakaunawaan.


## Simple Negate Rule #4:
#### upang -> upang hindi
#### para -> para hindi
#### para sa -> hindi para sa

In [12]:
def SimpleNegateUPANG(article):
  split_string = re.split(r'(?i)((?<!hindi)\s+para sa\s+(?!hindi|wala|walang)|(?<!hindi)\s+upang\s+(?!sa|hindi|wala|walang)|(?<!hindi)\s+para\s+(?!hindi|wala|walang))', article)
  print(split_string)

  for i in range(len(split_string)):
    if (split_string[i].casefold() == ' upang ') or (split_string[i].casefold() == ' para '):
      split_string[i] = split_string[i] + "hindi "
    elif (split_string[i].casefold() == ' para sa '):
      split_string[i] = " hindi" + split_string[i]
  return "". join(split_string)

test_string = "Isinasagawa ang survey upang mapabuti pa ang pagpapatakbo ng mga events sa susunod na pagkakataon. Ang biniling pagkain ay para sa inyo. Kailangan ito para matapos ang mga gawain. Para hindi mapagalitan, sila ay natulog nalang."
test_new = SimpleNegateUPANG(test_string)
print(test_new)

['Isinasagawa ang survey', ' upang ', 'mapabuti pa ang pagpapatakbo ng mga events sa susunod na pagkakataon. Ang biniling pagkain ay', ' para sa ', 'inyo. Kailangan ito', ' para ', 'matapos ang mga gawain. Para hindi mapagalitan, sila ay natulog nalang.']
Isinasagawa ang survey upang hindi mapabuti pa ang pagpapatakbo ng mga events sa susunod na pagkakataon. Ang biniling pagkain ay hindi para sa inyo. Kailangan ito para hindi matapos ang mga gawain. Para hindi mapagalitan, sila ay natulog nalang.


## Simple Negate Rule #5:
#### nang -> nang hindi

In [19]:
def SimpleNegateNANG(article):
  split_string = re.split(r'(?i)((?<!hindi)(?<!wala)\s+nang\s+(?!hindi|wala|walang|sa|may))', article)
  print(split_string)

  for i in range(len(split_string)):
    if (split_string[i].casefold() == ' nang '):
      if ((i > 0) and (i < len(split_string)-1)) and (split_string[i-1].casefold().split()[-1] == split_string[i+1].casefold().split()[0]):
        continue
      else:
        split_string[i] = split_string[i] + "hindi "
  return "". join(split_string)

test_string = "Kailangan nating pangalagaan ang kalikasan nang sa gayon ay hindi tayo nito parusahan. Nilagay niya nang paunti-unti ang mantika upang hindi ito tumalsik. Iyak nang iyak ang sanggol. "
test_new = SimpleNegateNANG(test_string)
print(test_new)

['Kailangan nating pangalagaan ang kalikasan nang sa gayon ay hindi tayo nito parusahan. Nilagay niya', ' nang ', 'paunti-unti ang mantika upang hindi ito tumalsik. Iyak', ' nang ', 'iyak ang sanggol. ']
Kailangan nating pangalagaan ang kalikasan nang sa gayon ay hindi tayo nito parusahan. Nilagay niya nang hindi paunti-unti ang mantika upang hindi ito tumalsik. Iyak nang iyak ang sanggol. 


## Simple Negate Rule #6:
#### kung saan -> kung saan hindi

In [20]:
def SimpleNegateKUNGSAAN(article):
  split_string = re.split(r'(?i)((?<!hindi)(?<!wala)\s+kung saan\s+(?!hindi|wala|walang|may))', article)
  print(split_string)

  for i in range(len(split_string)):
    if (split_string[i].casefold() == ' kung saan '):
      split_string[i] = split_string[i] + "hindi "
  return "". join(split_string)

test_string = "Pinuntahan ng pulisya ang lokasyon kung saan natagpuan ang bangkay. Sayang ang kanilang oras sa pagpunta sa tindahan kung saan wala naman ang bibilhin nila."
test_new = SimpleNegateKUNGSAAN(test_string)
print(test_new)

['Pinuntahan ng pulisya ang lokasyon', ' kung saan ', 'natagpuan ang bangkay. Sayang ang kanilang oras sa pagpunta sa tindahan kung saan wala naman ang bibilhin nila.']
Pinuntahan ng pulisya ang lokasyon kung saan hindi natagpuan ang bangkay. Sayang ang kanilang oras sa pagpunta sa tindahan kung saan wala naman ang bibilhin nila.
